In [1]:
import torch
from utils.experimentorUtils import getExperimentor
from utils.constants import *
print(torch.backends.cudnn.version())

8005


In [2]:
config = {
    "dataset_name": Dataset.CORA,
    "model_type": ModelType.GATV1,
    'num_of_epochs': 500,
    'num_of_runs': 3,
    'patience_period': 100,
    
    'batch_size': 1,
    'test_batch_size': 1,
    'num_workers': 2,
    'force_cpu': False,
    'test_frequency': 5,
    'console_log_freq': 1,
    'do_train_tqdm_logging': False,
    
    'lr': 0.005,
    'num_of_layers': 2, 
    'num_heads': 1,
    'hidden_size': 128,
    'dropout': 0.6,  
    "use_layer_norm": False,
    "use_batch_norm": False,
    
    'nbor_degree': 1,
    'adj_mode': AdjacencyMode.OneStep,
    'sparse': False
}

In [3]:
experimentor = getExperimentor(config["dataset_name"])(config)
experimentor.run_wrapper()

Run 01:


Epoch 01: 100%|██████████| 2708/2708 [00:26<00:00, 102.11it/s]


Epoch 01| Loss: 1.3104| Acc: 0.7068| Train Time: 26.5241s
Epoch 02| Loss: 1.1756| Acc: 0.7877| Train Time: 12.4341s
Epoch 03| Loss: 1.1136| Acc: 0.8205| Train Time: 12.7135s
Epoch 04| Loss: 1.0956| Acc: 0.8571| Train Time: 13.6720s
Epoch 05| Loss: 1.2769| Acc: 0.8353| Train Time: 14.8013s


Evaluating: 100%|██████████| 5416/5416 [00:17<00:00, 310.64it/s]

Train: 0.9714| Val: 0.9540| Test: 0.9490| Eval Time: 17.4388s
